> **Importing all libraries**

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

> **Importing complete data**

In [ ]:
raw_data = pd.read_csv("../input/logical-rhythm-2k21-cars24/train.csv")
raw_data

In [ ]:
raw_data.columns

> **Training(including both train + validation data) data processing**

In [ ]:
training_data = raw_data.drop(columns='price')
training_labels = raw_data["price"]

In [ ]:
training_data  = pd.get_dummies(training_data)
#training_data = training_data.drop(columns = 'pre_owner_4th Owner')
training_data = training_data.drop(columns = 'car ID')


In [ ]:
training_data

In [ ]:
training_labels

> **Spliting dataset in train_data and validation_data**

In [ ]:
train_data,validation_data,train_labels,validation_labels = train_test_split(training_data,training_labels,test_size = 0.1,random_state = 42)

> **Dataset visulaization**

In [ ]:
train_data

In [ ]:
train_labels

In [ ]:
plt.plot(raw_data["car ID"],raw_data["price"])
plt.ylabel('raw_data["price"]')
plt.xlabel('raw_data["car ID"]')

In [ ]:
raw_data["price"].plot(kind = "hist")

In [ ]:
plt.scatter(raw_data["car ID"],raw_data["price"])

In [ ]:
#4th owner cars in training data
raw_data[raw_data["pre_owner"]== "4th Owner"]

> **Normalization of train data and validation data**

In [ ]:
normalization = make_column_transformer((MinMaxScaler(),['model_year','distance_covered (km)']))
normalization.fit(train_data)
train_data_normalized = normalization.transform(train_data)
validation_data_normalized = normalization.transform(validation_data)

In [ ]:
train_data_normalized

In [ ]:
validation_data_normalized

> **Adding noramalized data to train_set and validation_set**

In [ ]:
train_data.columns
validation_data.columns

In [ ]:
train_data[['model_year','distance_covered (km)']] = train_data_normalized
validation_data[['model_year','distance_covered (km)']] = validation_data_normalized

In [ ]:
train_data

> **MODEL PREPARATION**

In [ ]:
#preparing model
tf.random.set_seed(42)
car_price_pridiction_model = tf.keras.Sequential([
                              tf.keras.layers.Dense(5000,activation = "relu"),
                              tf.keras.layers.Dense(5000,activation = "relu"),
                              tf.keras.layers.Dense(5000,activation = "relu"),
                              tf.keras.layers.Dropout(0.2),
                              
    
                              tf.keras.layers.Dense(4096,activation = "relu"),
                              tf.keras.layers.Dense(4096,activation = "relu"),
                              tf.keras.layers.Dense(4096,activation = "relu"), 
                              
                              
                              tf.keras.layers.Dense(2048,activation = "relu"),
                              tf.keras.layers.Dense(2048,activation = "relu"),
                              tf.keras.layers.Dense(2048,activation = "relu"),
                              
                              tf.keras.layers.Dense(1024,activation = "relu"),
                              tf.keras.layers.Dense(1024,activation = "relu"),
                              tf.keras.layers.Dense(1024,activation = "relu"),
                              
                              
                              tf.keras.layers.Dense(512,activation = "relu"),
                              tf.keras.layers.Dense(512,activation = "relu"),
                              tf.keras.layers.Dense(512,activation = "relu"),
                              
                              tf.keras.layers.Dense(256,activation = "relu"),
                              tf.keras.layers.Dense(256,activation = "relu"),
                              tf.keras.layers.Dense(256,activation = "relu"),
                              tf.keras.layers.Dense(128,activation = "relu"),
                              tf.keras.layers.Dense(128,activation = "relu"),
                              tf.keras.layers.Dense(1)
                                            ])
car_price_pridiction_model.compile(loss=tf.keras.losses.MeanSquaredLogarithmicError(),optimizer= tf.keras.optimizers.Adam(learning_rate = 0.0001))

car_price_pridiction = car_price_pridiction_model.fit(train_data,train_labels,batch_size = 32,steps_per_epoch = 50,epochs = 38,validation_data = (validation_data,validation_labels))


In [ ]:
pd.DataFrame(car_price_pridiction.history).plot()
plt.ylabel("loss")
plt.xlabel("epochs")

In [ ]:
car_price_pridiction_model.summary()

In [ ]:
# validation_labels.head(10)  # just for fun

In [ ]:
#car_price_pridiction_model.predict(validation_data)

In [ ]:
car_price_pridiction_model.evaluate(validation_data,validation_labels)

> **Importing testdata**

In [ ]:
raw_test_data = pd.read_csv("../input/logical-rhythm-2k21-cars24/test.csv")
raw_test_data

> **Preparing test dataset**

In [ ]:
test_data = pd.get_dummies(raw_test_data) 
test_data = test_data.drop(columns = 'car ID')
test_data

> **Normalization**

In [ ]:
normalization = make_column_transformer((MinMaxScaler(),['model_year','distance_covered (km)']))
normalization.fit(test_data)
test_data_normalized = normalization.transform(test_data)

> **Adding normalize data**

In [ ]:
# test_data = test_data.drop(columns = 'model_year')
# test_data = test_data.drop(columns = 'distance_covered (km)')
test_data[['model_year','distance_covered (km)']] = test_data_normalized


In [ ]:
test_data['4th Owner'] = 0

In [ ]:
test_data

In [ ]:
test_data.shape,train_data.shape

> **predicting over testdata set**

In [ ]:
price_pridiction = car_price_pridiction_model.predict(test_data)
price_pridiction

In [ ]:
raw_test_data["price"] = price_pridiction

> **Complete dataframe of testset with pridicted price**

In [ ]:
raw_test_data

In [ ]:
plt.plot(raw_data["car ID"],raw_data["price"])
plt.ylabel('raw_test_data["price"]')
plt.xlabel('raw_test_data["car ID"]')

> **CSV file of car id and price**

In [ ]:
car_id = raw_test_data["car ID"]
predicted_price = raw_test_data["price"]
dict = { "car id": car_id,
          "price":predicted_price}
predicted_submit_file = pd.DataFrame(dict)


In [ ]:
predicted_submit_file
predicted_submit_file.to_csv("submission_q2",index = False)